# Practice with two_moon data

In [ ]:
from PIL import Image

## ffmpeg image to video.
#ffmpeg -framerate 10 -i %4d_result.png -vcodec libx264 -pix_fmt yuv420p movie.mp4

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
%matplotlib inline

N = 100
K = 2

def one_hot_coding(y):
    k,n = np.max(y)+1, np.shape(y)[0]
    one_hot_y = np.zeros([n,k])
    one_hot_y[np.arange(n),y] = 1
    return one_hot_y

def class_number(y):
    n = np.shape(y)[0]
    y_number = np.argmax(y, axis=1) 
    return y_number

def shape_scatter(X, y, weights, biases):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    assert len(weights) == len(biases), "error"
    
    for i,w in enumerate(weights):
        if i==0:
            Z = np.dot(np.c_[xx.ravel(), yy.ravel()], weights[i])+biases[i]
        else:
            Z = np.dot(np.maximum(0, Z), weights[i])+biases[i]
    
    Z = np.argmax(Z, axis=1)
    Z = Z.reshape(xx.shape)
    fig = plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
    plt.show()
def shape_save(X, y, weights, biases, iteration):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    assert len(weights) == len(biases), "error"
    
    for i,w in enumerate(weights):
        if i==0:
            Z = np.dot(np.c_[xx.ravel(), yy.ravel()], weights[i])+biases[i]
        else:
            Z = np.dot(np.maximum(0, Z), weights[i])+biases[i]
    
    Z = np.argmax(Z, axis=1)
    Z = Z.reshape(xx.shape)
    fig = plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
    fig.savefig('./picture/'+str(format(iteration, '04'))+'_result.png')
    plt.show()
    plt.close(fig)
    print(iteration, 'th saved.')
    
def scatter(X, y):
    if len(y.shape) > 1: # one-hot coding
        y = class_number(y)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.show()

# # lets visualize the data:    
X, y = make_moons(n_samples=N, noise=0.1)
y= one_hot_coding(y)
print(X.shape)

scatter(X,y)

In [ ]:
def accuracy(y_true, y_pred):
    """
    accuracy 구하기

    Args:
        y_true: true label
        y_hat: predicted label
    Returns:
        Accuracy
    Hints
        - tf.equal : 값을 비교하여 동일하면 1, 다르면 0을 return
        - tf.argmax : 값이 최대인 위치의 index를 return
        - tf.cast : data type을 변경하는 함수
        - tf.reduce_mean : 지정한 축(axis)에 대해 평균을 취하는 함수.
    """
    with tf.name_scope('accuracy'):
        prediction = tf.argmax(y_pred, axis=1)
        true = tf.argmax(y_true, axis=1)
        acc = tf.reduce_mean(tf.cast(tf.equal(true, prediction), 'float'))
    return acc

## Linear Classifier (선형 분류기)

## <center> \\( f(x) = xW+b \\)</center>

In [ ]:
def linear_classifier(x, out_dim, name):
    """
    Args :
        Inputs : Input tensor
        out_dim : output dimension
        name : name for parameter
    Returns:
        output
        [weights] : list
        [biases] : list
    Hints
        - tf.Variable  
        - tf.matmul
    """
    
    with tf.variable_scope(name):
        in_dim = x.get_shape().as_list()[1]
        w = tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=0.01), tf.float32)
        b = tf.Variable(tf.zeros(out_dim), tf.float32)
        output = tf.add(tf.matmul(x, w),b)
    return output, [w], [b]

## Loss function (손실 함수) : Cross Entropy

# <center> \\( L(y_i, f(x_i; W)) = -\frac{1}{n}\sum_{i=1}^{n} y_{i} log(f(x_i))\\)</center>

In [ ]:
def cross_entropy(y_true, y_pred, epsilon=1e-7):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    Hints
        - tf.reduce_mean  
        - tf.reduce_sum
        - tf.log
    """
    with tf.name_scope('cross_entropy'):
        loss = -tf.reduce_mean(tf.reduce_sum(y_true*tf.log(y_pred+epsilon), axis=1))
    return loss

# 1. With Softmax Linear Classifier
## <center> \\( f(x_i; W,b) =  \sigma(xW+b) \\)</center>

> where $\sigma$ is softmax function, i.e.,

## <center> \\( \sigma(x)_i =  \frac{e^{x_i}}{\sum_j e^{x_j}}  \\)</center>

In [ ]:
learning_rate = 1e-2
iteration = 100
step_size = iteration/10

In [ ]:
def model_1(x, k):
    """
        Softmax Linear Classifier Model
        
        Args:
            x : input
            k : the number of classes, i.e., output dimension of the model
        Returns:
            output
            weights list
            bias list
        Hints:
            tf.nn.softmax : softmax function
    """
    out, w, b = linear_classifier(x, k, 'fc1')
    out = tf.nn.softmax(out)
    return out, w, b

In [ ]:
tf.reset_default_graph()
# ---------------------------------------------------------------------------------------------------------
# 1) Declare Placeholders
# input, true_label을 위한 placeholder 선언
# Hint : tf.placeholder, input과 ouput이 어떤 차원을 가져야할지 고민해봅시다.
# ---------------------------------------------------------------------------------------------------------
x = tf.placeholder(tf.float32, [None, 2], 'x')
y_true = tf.placeholder(tf.float32, [None, K], 'y')

# ---------------------------------------------------------------------------------------------------------
# 2) model declare
# Prediction을 위해 기존의 정의한 model을 사용합니다.
# ---------------------------------------------------------------------------------------------------------
y_pred, w, b = model_1(x, K)

# ---------------------------------------------------------------------------------------------------------
# 3) obtain loss & accuracy
# 앞서 정의한 function들을 통해 loss와 accuracy를 얻습니다.
# ---------------------------------------------------------------------------------------------------------
loss = cross_entropy(y_true, y_pred)
acc = accuracy(y_true, y_pred)

# ---------------------------------------------------------------------------------------------------------
# 4) Optimizer
# Parameter Update를 위한 Optimizer를 선언합니다. Learning rate는 learning_rate로 선언된 변수를 사용합니다.
# Hint : tf.train.GradientDescentOptimizer
# ---------------------------------------------------------------------------------------------------------
train_ops = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# ---------------------------------------------------------------------------------------------------------
# 7) Session 
# tensorflow의 연산을 수행하기 위해 session을 호출합니다.
# 8) variable iniializer 
# tensorflow의 variable들을 초기화 하기 위해 initializer를 사용합니다.
# Hint : tf.global_variables_initializer
# 9) training 
# 학습에서 사용할 iteration 수는 위의 iteration을 사용합니다. Loss 및 accuracy를 check할 때 step_size를 사용합니다.
# ---------------------------------------------------------------------------------------------------------
j = 0
with tf.Session() as sess:
    train_x, train_y = X,y
    sess.run(tf.global_variables_initializer())
    for i in range(iteration):
        _,loss_ = sess.run([train_ops, loss], feed_dict={x:train_x, y_true:train_y})  
        if (i+1)%step_size == 0:
            pred, acc_, weights, biases = sess.run([y_pred, acc, w, b], feed_dict={x:train_x, y_true:train_y})
            pred = np.argmax(pred, axis=1)
            print("{} iteration, loss : {:.4f}, accuracy : {:.4f}".format(i+1, loss_, acc_))
            scatter(train_x, pred)
            shape_scatter(train_x, pred, weights, biases)

# 2. With Neural Network (Fully Connected Layer)

In [ ]:
def model_2(x, k):
    """
        Neural Network with 2 Hidden layer.
        Args:
            x : input
            k : the number of classes, i.e., output dimension of the model
        Returns:
            output
            weights list
            bias list
        Hint:
            tf.nn.relu : activation function
            tf.nn.softmax : softmax function
    """
    
    out, w1, b1 = linear_classifier(x, 100, 'fc1')
    out = tf.nn.relu(out)
    out, w2, b2= linear_classifier(out, 100, 'fc2')
    out = tf.nn.relu(out)
    out, w3, b3 = linear_classifier(out, k, 'fc3')
    out = tf.nn.softmax(out)
    return out, w1+w2+w3, b1+b2+b3

In [ ]:
N=200
K=2
learning_rate = 1e-0
iteration = 240
l = 0
step_size = iteration/10

In [ ]:
from IPython.display import clear_output
import time

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 2], 'x')
y_true = tf.placeholder(tf.float32, [None, K], 'y')

y_pred, w, b = model_2(x, K)
loss = cross_entropy(y_true, y_pred)
acc = accuracy(y_true, y_pred)

train_ops = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
j=1
with tf.Session() as sess:
    train_x, train_y = X,y
    sess.run(tf.global_variables_initializer())
    for i in range(iteration):
        _,loss_ = sess.run([train_ops, loss], feed_dict={x:train_x, y_true:train_y})  
        if (i+1)%2 == 0:
            pred, acc_, weights, biases = sess.run([y_pred, acc, w, b], feed_dict={x:train_x, y_true:train_y})
            pred = np.argmax(pred, axis=1)
            shape_scatter(train_x, pred, weights, biases)


# 3. With Your work

In [ ]:
N = 100
K = 5

In [ ]:
def model_3(x, k):
    """
        Neural Network with 2 Hidden layer.
        Args:
            x : input
            k : the number of classes, i.e., output dimension of the model
        Returns:
            output
            weights list
            bias list
        Hint:
            tf.nn.relu : activation function
            tf.nn.softmax : softmax function
    """
    
    out, w1, b1 = linear_classifier(x, 100, 'fc1')
    out = tf.nn.relu(out)
    out, w2, b2= linear_classifier(out, 100, 'fc2')
    out = tf.nn.relu(out)
    out, w3, b3= linear_classifier(out, 100, 'fc3')
    out = tf.nn.relu(out)
    out, w4, b4 = linear_classifier(out, k, 'fc4')
    out = tf.nn.softmax(out)
    return out, w1+w2+w3+w4, b1+b2+b3+b4

In [ ]:
learning_rate = 1e-0
iteration = 1000
l = 3e-4
step_size = iteration/10

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 2], 'x')
y_true = tf.placeholder(tf.float32, [None, K], 'y')

y_pred, w, b = model_3(x, K)
loss = cross_entropy(y_true, y_pred)
acc = accuracy(y_true, y_pred)

train_ops = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    train_x, train_y = spiral_data(n=N, k=K)
    sess.run(tf.global_variables_initializer())
    for i in range(iteration):
        _,loss_ = sess.run([train_ops, loss], feed_dict={x:train_x, y_true:train_y})  
        if (i+1)%100 == 0:
            clear_output()
            pred, acc_, weights, biases = sess.run([y_pred, acc, w, b], feed_dict={x:train_x, y_true:train_y})
            pred = np.argmax(pred, axis=1)
            print("{} iteration, loss : {:.4f}, accuracy : {:.4f}".format(i+1, loss_, acc_))
            shape_scatter(train_x, pred, weights, biases)
            time.sleep(0.5)